In [16]:
# 使用sys.path添加上级目录
import sys
import os
package_path = os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(package_path, "ch05", "06_user_interface")
print(file_path)
sys.path.append(file_path)

import torch
if torch.cuda.is_available():
   device = torch.device("cuda")
elif torch.backends.mps.is_available():
   device = torch.device("mps")
else:
   device = torch.device("cpu")

/Users/young/project/llmProject/LLMs-from-scratch-CN/ch05/06_user_interface


In [17]:
# ! pip3 install sentencepiece

In [18]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",
    "sentencepiece",
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

huggingface_hub version: 0.29.2
sentencepiece version: 0.2.0
torch version: 2.6.0


# 1.逐步转换GPT模型的实现

## 1.1 替换LayerNorm为RMSNorm层

In [19]:
import torch
import torch.nn as nn

#####################################
# Chapter 4
#####################################

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x):
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift

class RMSNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.emb_dim = emb_dim
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()

    def forward(self, x):
        means = x.pow(2).mean(dim=-1, keepdim=True)
        x_normed = x * torch.rsqrt(means + self.eps)
        return (x_normed * self.weight).to(dtype=x.dtype)

In [20]:
torch.manual_seed(123)

# [batch_size, seq_len, emb_dim]
example_batch = torch.randn(2, 3, 4)

rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)

print(rms_norm(example_batch))
print("*" * 30)
print(rmsnorm_pytorch(example_batch))


assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))

tensor([[[ 0.8834, -0.4655, -0.7948, -1.5398],
         [ 0.8053,  1.5254, -0.5074, -0.8759],
         [-0.1619, -0.4049,  0.7016, -1.8214]],

        [[ 1.3348, -0.1232,  0.9638, -1.1288],
         [-0.8511, -1.5285, -0.4420,  0.8624],
         [ 0.9398,  0.2082,  1.7241, -0.3180]]], grad_fn=<MulBackward0>)
******************************
tensor([[[ 0.8834, -0.4655, -0.7948, -1.5398],
         [ 0.8053,  1.5254, -0.5074, -0.8759],
         [-0.1619, -0.4049,  0.7016, -1.8214]],

        [[ 1.3348, -0.1232,  0.9638, -1.1288],
         [-0.8511, -1.5285, -0.4420,  0.8624],
         [ 0.9398,  0.2082,  1.7241, -0.3180]]], grad_fn=<MulBackward0>)


## 1.2 用SiLU代替GELU作激活函数

In [21]:
#####################################
# Chapter 4
#####################################

# class GELU(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         return 0.5 * x * (1 + torch.tanh(
#             torch.sqrt(torch.tensor(2.0 / torch.pi)) *
#             (x + 0.044715 * torch.pow(x, 3))
#         ))

class SiLU(nn.Module):
    def __init__(self):
        super(SiLU, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

In [22]:
silu = SiLU()

assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

## 1.3 更新FeedForward模块

In [23]:
#####################################
# Chapter 4
#####################################
# class FeedForward(nn.Module):
#     def __init__(self, cfg):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
#             GELU(),
#             nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
#         )

#     def forward(self, x):
#         return self.layers(x)

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()
    
    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        x = self.silu(x_fc1) * x_fc2
        return self.fc3(x)

## 1.4 实现RoPE旋转位置编码

In [24]:
# 预计算RoPE参数
def precompute_rope_params(head_dim, theta_base=10_000, context_length=1024):
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # 计算逆频率
    inv_freq = 1.0 / (theta_base **(torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # 生成位置索引
    positions = torch.arange(context_length)

    # 计算旋转角度
    # 通过维度扩展+广播实现
    # positions: [context_length] -> [context_length, 1]
    # inv_freq: [head_dim // 2] -> [1, head_dim // 2]
    # angles: [context_length, head_dim // 2]
    angles = positions[:, None] * inv_freq[None, :]

    # (context_length, head_dim)
    angles = torch.cat([angles, angles], dim=1)

    # 计算正弦和余弦值
    # [context_length, head_dim // 2]
    sin = torch.sin(angles)
    cos = torch.cos(angles)

    return cos, sin

# 计算RoPE
def compute_rope(x, cos, sin):
    # x: (batch_size, num_heads, seq_len, head_dim)
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 == 0, "Head dimension must be even"

    # 将x划分为两部分, 分为偶数项和奇数项
    x1 = x[..., : head_dim // 2]
    x2 = x[..., head_dim // 2 :]

    # shape: (1, 1, seq_len, head_dim)
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # 应用旋转变换
    rotated = torch.cat((-x2, x1), dim=-1)
    x_rotated = (x * cos) + (rotated * sin)

    return x_rotated.to(dtype=x.dtype)

In [25]:
# 示例：将rope应用于q和k
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

# 初始化
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# 构建测试用q和k
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.randn(batch_size, num_heads, context_len, head_dim)

# 应用rope到pos emb
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)


## 1.5 在`MultiHeadAttention`模块中添加RoPE

In [26]:
# 基于Chapter 3的多头注意进行修改
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        ################################### NEW ###################################
        # 不同点：Set bias=False and dtype=dtype for all linear layers below
        ########################################################################### 
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)
        ################################### NEW ###################################
        # 不同点：去除dropout
        ########################################################################### 
        # self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        # 不同点：使用rope
        ########################################################################### 
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # (batch_size, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        
        # (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        ################################### NEW ###################################
        # 不同点：使用rope
        ########################################################################### 
        keys = compute_rope(keys, self.cos, self.sin)
        queries = compute_rope(queries, self.cos, self.sin)

        # attn score
        # (b, num_heads, num_tokens, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)

        # causal mask
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # attn weight
        # (b, num_heads, num_tokens, num_tokens)
        attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)

        # context vec
        # (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # (b, num_tokens, d_out)
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec        


In [27]:
# 测试MultiHeadAttention模块
batch_size = 1
context_len = 100
max_context_len = 4096
embed_dim = 128
num_heads = 4

example_batch = torch.randn((batch_size, context_len, embed_dim))

mha = MultiHeadAttention(
    d_in=embed_dim,
    d_out=embed_dim,
    context_length=max_context_len,
    num_heads=num_heads
)

mha(example_batch)

del mha

## 1.6 更新TransformerBlock模块
  - 将 **LayerNorm** 替换为 **RMSNorm**  
  - 移除 **dropout**  
  - 移除 **`qkv_bias` 设置**  
  - 添加 **`dtype` 设置**

In [28]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dtype=cfg["dtype"] # NEW
            # dropout=cfg["drop_rate"],
            # qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)

        ################################### NEW ###################################
        # self.norm1 = LayerNorm(cfg["emb_dim"])
        # self.norm2 = LayerNorm(cfg["emb_dim"])
        self.norm1 = RMSNorm(cfg["emb_dim"])
        self.norm2 = RMSNorm(cfg["emb_dim"])

        # self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        # x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # x = self.drop_shortcut(x)
        x = x + shortcut

        return x

## 1.7 更新模型类

In [29]:
class Llama2Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
        # self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        ################################### NEW ###################################
        # self.final_norm = LayerNorm(cfg["emb_dim"])
        self.final_norm = RMSNorm(cfg["emb_dim"])
        ###########################################################################

        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

    def forward(self, in_idx):
        # batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds # + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

# 2. 模型初始化

In [30]:
# 模型初始化
LLAMA2_CONFIG_7B = {
    "vocab_size": 32000,     # Vocabulary size
    "context_length": 4096,  # Context length
    "emb_dim": 4096,         # Embedding dimension
    "n_heads": 32,           # Number of attention heads
    "n_layers": 32,          # Number of layers
    "hidden_dim": 11008,     # NEW: Size of the intermediate dimension in FeedForward
    "dtype": torch.bfloat16  # NEW: Lower-precision dtype to reduce memory usage
}

In [31]:
model = Llama2Model(LLAMA2_CONFIG_7B)

In [32]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 6,738,415,616


In [33]:
def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    # 统计模型参数量
    for param in model.parameters():
        param_size = param.numel()
        total_params += param_size
        if param.requires_grad:
            total_grads += param_size

    # 计算buffer size(register_buffer，不参与梯度更新)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # 计算模型内存大小
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # 转换为GB
    total_memory_gb = total_memory_bytes / (1024 ** 3)

    return total_memory_gb

print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

float32 (PyTorch default): 52.33 GB
bfloat16: 26.17 GB


In [34]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

model.to(device);

# 3. 加载tokenizer

In [35]:
# !pip3 install huggingface_hub sentencepiece

In [36]:
# from huggingface_hub import login
# import json

# with open("config.json", "r") as config_file:
#     config = json.load(config_file)
#     access_token = config["HF_ACCESS_TOKEN"]

# login(token=access_token)

In [37]:
# from huggingface_hub import whoami

# try:
#     user_info = whoami()
#     print(f"已登录为: {user_info['name']}")
# except Exception as e:
#     print(f"未登录或登录已过期: {e}")

In [38]:
# # 下载词汇表
# from huggingface_hub import hf_hub_download

# tokenizer_file = hf_hub_download(
#     repo_id="meta-llama/Llama-2-7b",
#     filename="tokenizer.model",
#     local_dir="Llama-2-7b"
# )

In [39]:
# ! pip3 install modelscope

In [40]:
# from modelscope import snapshot_download

# # 只下载特定文件（如分词器）
# model_dir = snapshot_download('shakechen/Llama-2-7b-hf', 
#                             #  file_pattern='tokenizer.model',
#                              cache_dir='./Llama-2-7b'
#                              )

# print(f"模型已下载到: {model_dir}")

In [41]:
from modelscope.hub.file_download import model_file_download

# 下载特定文件
tokenizer_file = model_file_download(
    model_id='shakechen/Llama-2-7b-hf',
    file_path='tokenizer.model',
    local_dir='./Llama-2-7b'
)

In [42]:
import sentencepiece as spm

class LlamaTokenizer:
    def __init__(self, tokenizer_file):
        sp = spm.SentencePieceProcessor()
        sp.load(tokenizer_file)
        self.tokenizer = sp
    
    def encode(self, text):
        return self.tokenizer.encode_as_ids(text)
    
    def decode(self, ids):
        return self.tokenizer.decode_pieces(ids)

tokenizer = LlamaTokenizer(tokenizer_file)

In [43]:
from previous_chapters import generate, text_to_token_ids, token_ids_to_text

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves", tokenizer).to(device),
    max_new_tokens=30,
    context_size=LLAMA2_CONFIG_7B["context_length"],
    top_k=1,
    temperature=0.
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort movesαllRadius deletingpretcc否']; future eer napulate lackус während inter DES издаSchéonkkarto Оryptato#{ningproof eerbye


# 4. 加载预训练权重

In [45]:
weights_file_1 = model_file_download(
    model_id='shakechen/Llama-2-7b-hf',
    file_path='pytorch_model-00002-of-00002.bin',
    local_dir='./Llama-2-7b'
)

In [46]:
weights_file_2 = model_file_download(
    model_id='shakechen/Llama-2-7b-hf',
    file_path='pytorch_model-00001-of-00002.bin',
    local_dir='./Llama-2-7b'
)

In [48]:
# 创建空的state_dict
state_dict = {}

# 加载所有分片
shard_files = [
    "pytorch_model-00001-of-00002.bin", 
    "pytorch_model-00002-of-00002.bin"
]

for shard_file in shard_files:
    # 加载分片
    shard_state_dict = torch.load(
        os.path.join("./Llama-2-7b", shard_file), 
        map_location=device  # 根据需要指定设备
    )
    # 合并到主state_dict
    state_dict.update(shard_state_dict)

RuntimeError: MPS backend out of memory (MPS allocated: 27.16 GB, other allocations: 1.70 MB, max allowed: 27.20 GB). Tried to allocate 250.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
weights = torch.load(state_dict, weights_only=True)

In [ ]:
list(weights.keys())[:15]

In [ ]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")

    if isinstance(right, torch.Tensor):
        return torch.nn.Parameter(right.clone().detach())
    else:
        return torch.nn.Parameter(torch.tensor(right))


def load_weights_into_llama(model, param_config, params):
    model.tok_emb.weight = assign(model.tok_emb.weight, params["tok_embeddings.weight"])

    for l in range(param_config["n_layers"]):

        # Load attention weights
        model.trf_blocks[l].att.W_query.weight = assign(
            model.trf_blocks[l].att.W_query.weight,
            params[f"layers.{l}.attention.wq.weight"]
        )
        model.trf_blocks[l].att.W_key.weight = assign(
            model.trf_blocks[l].att.W_key.weight,
            params[f"layers.{l}.attention.wk.weight"]
        )
        model.trf_blocks[l].att.W_value.weight = assign(
            model.trf_blocks[l].att.W_value.weight,
            params[f"layers.{l}.attention.wv.weight"]
        )
        model.trf_blocks[l].att.out_proj.weight = assign(
            model.trf_blocks[l].att.out_proj.weight,
            params[f"layers.{l}.attention.wo.weight"]
        )
        model.trf_blocks[l].norm1.weight = assign(
            model.trf_blocks[l].norm1.weight,
            params[f"layers.{l}.attention_norm.weight"]
        )

        # Load FeedForward weights
        model.trf_blocks[l].ff.fc1.weight = assign(
            model.trf_blocks[l].ff.fc1.weight,
            params[f"layers.{l}.feed_forward.w1.weight"]
        )
        # For some reason w2 and w3 are provided in the wrong order in the weights file
        model.trf_blocks[l].ff.fc2.weight = assign(
            model.trf_blocks[l].ff.fc2.weight,
            params[f"layers.{l}.feed_forward.w3.weight"]
        )
        model.trf_blocks[l].ff.fc3.weight = assign(
            model.trf_blocks[l].ff.fc3.weight,
            params[f"layers.{l}.feed_forward.w2.weight"]
        )
        model.trf_blocks[l].norm2.weight = assign(
            model.trf_blocks[l].norm2.weight,
            params[f"layers.{l}.ffn_norm.weight"]
        )

    # Load output layer weights
    model.final_norm.weight = assign(model.final_norm.weight, params["norm.weight"])
    model.out_head.weight = assign(model.out_head.weight, params["output.weight"])


load_weights_into_llama(model, LLAMA2_CONFIG_7B, weights)
model.to(device);

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort", tokenizer).to(device),
    max_new_tokens=25,
    context_size=LLAMA2_CONFIG_7B["context_length"],
    top_k=1,
    temperature=0.
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

# 5. 使用指令微调模型

In [ ]:
del model

In [ ]:
weights_file_1 = model_file_download(
    model_id='shakechen/Llama-2-7b-chat-hf',
    file_path='pytorch_model-00002-of-00002.bin',
    local_dir='./Llama-2-7b-chat'
)

In [ ]:
weights_file_2 = model_file_download(
    model_id='shakechen/Llama-2-7b-chat-hf',
    file_path='pytorch_model-00001-of-00002.bin',
    local_dir='./Llama-2-7b-chat'
)

In [ ]:
model = Llama2Model(LLAMA2_CONFIG_7B)
load_weights_into_llama(model, LLAMA2_CONFIG_7B, weights)
model.to(device);

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("What do llamas eat?", tokenizer).to(device),
    max_new_tokens=25,
    context_size=LLAMA2_CONFIG_7B["context_length"],
    top_k=1,
    temperature=0.
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))